In [1]:
import psycopg
import simplejson
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import pandas as pd

In [2]:
def gen_raport(conn):
    
    cursor = conn.cursor()
    
    query = """
    SELECT pomiary.pomiar, pomiary.data, adresy.ulica, adresy.numer_budynku, adresy.numer_mieszkania
    FROM pomiary
    JOIN adresy ON pomiary.id_adresu = adresy.id;
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    cursor.close()
    
    df = pd.DataFrame(rows, columns=['pomiar', 'data', 'ulica', 'numer_budynku', 'numer_mieszkania'])

    plt.figure(figsize=(10, 6))
    plt.hist(df['pomiar'], bins=10, alpha=0.5, label='Wszystkie adresy')
    plt.xlabel('Pomiar')
    plt.ylabel('Liczebność')
    plt.title('Histogram pomiarów dla wszystkich adresów')
    plt.legend()
    plt.tight_layout()
    
    plt.show()
    
    df = pd.DataFrame(rows, columns=['pomiar', 'data', 'ulica', 'numer_budynku', 'numer_mieszkania'])

    df_grouped = df.groupby('ulica').size().reset_index(name='count')

    plt.figure(figsize=(10, 6))
    plt.bar(df_grouped['ulica'], df_grouped['count'])
    plt.xlabel('Ulica')
    plt.ylabel('Ilość pomiarów')
    plt.title('Wykres słupkowy - Ilość pomiarów dla poszczególnych ulic')
    plt.xticks(rotation=90)

    plt.show()


In [3]:
def gen_dataframe(data,sql_columns):
    columns_name = []
    for el in sql_columns:
        match el:
            case 'pomiary.pomiar':
                columns_name.append("Pomiar")
            case 'pomiary.data':
                columns_name.append("Data pomiaru")
            case 'adresy.ulica':
                columns_name.append("Ulica")
            case 'adresy.numer_budynku':
                columns_name.append("Numer budynku")
            case 'adresy.numer_mieszkania':
                columns_name.append("Numer mieszkania")
    df = pd.DataFrame(data, columns=columns_name)
    print(df)
    print()
    print("Zapisać dane do csv?")
    wybor = input("1 - TAK, 0 - NIE")
    print()
    if(wybor == '1'):
        df.to_csv('dane.csv', index=False)
    

In [4]:
def select_data(conn,sql_columns,sql_condition):
    cur = conn.cursor()
    query = f"SELECT {', '.join(sql_columns)} FROM adresy JOIN pomiary ON adresy.id = pomiary.id_adresu WHERE {sql_condition}"
    cur.execute(query)
    results = cur.fetchall()
    gen_dataframe(results,sql_columns)

In [5]:
def prepare_conditions(conn,kolumny,warunki):
    sql_columns = []
    sql_condition = ""
    i=0
    for el in kolumny:
        match el:
            case '1':
                sql_columns.append("pomiary.pomiar")
            case '2':
                sql_columns.append("pomiary.data")
            case '3':
                sql_columns.append("adresy.ulica")
            case '4':
                sql_columns.append("adresy.numer_budynku")
            case '5':
                sql_columns.append("adresy.numer_mieszkania")
    for el in warunki:
        if(i>0):
            sql_condition += " AND "
        match el:
            case '1':
                print("1. Większy niż")
                print("2. Mniejszy niż")
                print("3. Równy")
                wybor = input("Pomiar ma być: ")
                wartosc = input("Podaj wartość do warunku: ")
                print()
                match wybor:
                    case '1':
                        sql_condition += "pomiary.pomiar > " + wartosc
                    case '2':
                        sql_condition += "pomiary.pomiar < " + wartosc
                    case '3':
                        sql_condition += "pomiary.pomiar = " + wartosc
            case '2':
                print("1. Większy niż")
                print("2. Mniejszy niż")
                print("3. Równy")
                wybor = input("Data pomiaru ma być: ")
                wartosc = input("Podaj wartość do warunku: ")
                print()
                match wybor:
                    case '1':
                        sql_condition += "pomiary.data > " + "'" + wartosc + "'"
                    case '2':
                        sql_condition += "pomiary.data < " + "'" + wartosc + "'"
                    case '3':
                        sql_condition += "pomiary.data = " + "'" + wartosc + "'"
            case '3':
                wartosc = input("Podaj nazwę ulicy: ")
                print()
                sql_condition += "adresy.ulica = " + "'" + wartosc + "'"
            case '4':
                print("1. Większy niż")
                print("2. Mniejszy niż")
                print("3. Równy")
                wybor = input("Numer budynku ma być: ")
                wartosc = input("Podaj wartość do warunku: ")
                print()
                match wybor:
                    case '1':
                        sql_condition += "adresy.numer_budynku > " + wartosc
                    case '2':
                        sql_condition += "adresy.numer_budynku < " + wartosc
                    case '3':
                        sql_condition += "adresy.numer_budynku = " + wartosc
            case '5':
                print("1. Większy niż")
                print("2. Mniejszy niż")
                print("3. Równy")
                wybor = input("Numer mieszkania ma być: ")
                wartosc = input("Podaj wartość do warunku: ")
                print()
                match wybor:
                    case '1':
                        sql_condition += "adresy.numer_mieszkania > " + wartosc
                    case '2':
                        sql_condition += "adresy.numer_mieszkania < " + wartosc
                    case '3':
                        sql_condition += "adresy.numer_mieszkania = " + wartosc
                
        i += 1
    select_data(conn,sql_columns,sql_condition)

In [6]:
def program(conn,wybor):
    match wybor:
        case '1':
            gen_raport(conn)
        case '2':
            kolumny = []
            warunki = []
            while True:
                print("0. Dalej")
                print("1. Pomiar")
                print("2. Date pomiaru")
                print("3. Ulice")
                print("4. Numer budynku")
                print("5. Numer mieszkania")
                wybor = input("Wybierz jakie kolumny wyświetlić: ")
                print()
                if(wybor== '0'):
                    break
                kolumny.append(wybor)
            while True:
                print("0. Dalej")
                print("1. Pomiar")
                print("2. Date pomiaru")
                print("3. Ulice")
                print("4. Numer budynku")
                print("5. Numer mieszkania")
                wybor = input("Wybierz po jakich kolumnach filtrować dane: ")
                print()
                if(wybor== '0'):
                    break
                warunki.append(wybor)
                
            prepare_conditions(conn,kolumny,warunki)
    

In [7]:
def main():
    with open("database_creds.json") as db_con_file:
        creds = simplejson.loads(db_con_file.read())
   
    conn = psycopg.connect(
        host=creds['host_name'],
        user=creds['user_name'],
        dbname=creds['db_name'],
        password=creds['password'],
        port=creds['port_number'])
    
    while True:
        print("0. Wyjdź")
        print("1. Wyświetl raport")
        print("2. Wyświetl dane")
        wybor = input("Wybierz opcje: ")
        print()
        if(wybor == '0'):
            break
        program(conn,wybor)
    
    conn.close()


In [ ]:
if __name__ == '__main__':
    main()

0. Wyjdź
1. Wyświetl raport
2. Wyświetl dane
